In [1]:
# import dependencies
import pandas as pd
import datetime
from dateutil import parser


In [2]:
# read csv with data for open mic events
OMs = pd.read_csv('resources/data/TCB_mics_calendar - TCB_mics_calendar copy 6.csv')

# remove rows that don't have a name or a venue
OMs = OMs.dropna(subset=['EVENT NAME', 'VENUE'])

# dictionary with adjusted column names
columns = {}

# list with columns to be dropped
drops = []

# adjust column names and determine which columns should be dropped
for col in OMs.columns:
    c = col.replace(' - MIC', '').replace('CATEGORIES', 'CATEGORY').replace('?', '').replace(' MICS', '').replace(' ', '_').lower()
    if '(dont_fill)' in c:
        drops.append(c)
    columns[col] = c

# set column names to adjusted column names
OMs = OMs.rename(columns=columns)

# remove columns that aren't supposed to be filled
OMs = OMs.drop(columns=drops)

# display dataframe
OMs


,event_name,venue,neighborhood,start_date,start_time,end_date,end_time,all_day_event,category,event_cost,purchase_req,free,show_map_link,show_map,event_description,stage_time,event_website
0,(11AM-12PM Hourly) Improv Mic @ Fourth Wall Co...,Fourth Wall Comedy Cafe,Hollywood/East Hollywood,2020-03-15,11:00 AM,2020-03-15,1:45 PM,False,Open Mic,$5,No min.,NaN,True,True,For stand-up riffing and crowd work\nSuggestio...,5 min.,https://slotted.co/fwcafe
1,Improv Mic,The Hollywood Comedy,Hollywood/East Hollywood,2020-03-15,11:00 AM,2020-03-15,12:00 PM,False,Improvised stand-up mic,$5,No min.,NaN,True,True,includes snacks and drinks\nSign-up in advance...,5 min.,https://slotted.co/thehollywoodcomedy
2,(1PM-6PM Hourly) The Hollywood Comedy,The Hollywood Comedy,Hollywood/East Hollywood,2020-03-08,1:00 PM,2020-03-08,7:00 PM,False,Open Mic,$5,No min.,NaN,True,True,includes snacks & drinks\nSign-up in advance a...,5 min.,https://slotted.co/thehollywoodcomedy
3,(12PM-1PM Hourly) Flashback Mic (in Melrose),Flashback Mic (on Melrose),WeHo/Fairfax/Beverly Hills,2020-03-15,12:00 PM,2020-03-15,2:00 PM,False,Open Mic,$5,No min.,NaN,True,True,"*entrance located in rear, look for the red do...",5 min.,https://slotted.co/fbmic
4,(1PM-12AM Hourly) 15 Mins. The Hollywood Comedy,The Hollywood Comedy,Hollywood/East Hollywood,2020-03-08,1:00 PM,2020-03-08,11:59 PM,False,Open Mic,$12,No min.,NaN,True,True,"\n\nin ""The Fancy Room"" (up front)\n$12 (inclu...",15 min.,https://slotted.co/thehollywoodcomedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,Geeky Teas & Games,Geeky Teas & Games,Burbank/NoHo/Valley,2020-02-22,9:30 PM,2020-02-22,11:30 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes bottled water\nSign-up 9PM\nStart...,5 min.,NaN
457,Santa Monica Playhouse,Santa Monica Playhouse,Westside/Santa Monica,2020-02-29,9:30 PM,2020-02-29,12:00 AM,False,Open Mic,$5,No min.,NaN,True,True,"\n\nNot on Main Stage, in Other Theatre\nPassw...",5 min.,NaN
458,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-14,10:00 PM,2020-03-14,11:00 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
459,Atomic Wombat Comics,Atomic Wombat Comics,Orange County,2020-03-07,10:30 PM,2020-03-07,11:30 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nSign-up in advance at https://slotted.co/c...,5 min.,https://slotted.co/comedygrabbag


In [3]:
# list to hold columns for start and end dates/times
begin_and_end = ['start_date', 'end_date', 'start_time', 'end_time']

# convert data in these columns from strings to timestamps
for i , t in enumerate(begin_and_end):
    if i < 2:
        OMs[t] = OMs[t].apply(lambda x: parser.parse(x))
    else:
        OMs[t] = OMs[t].apply(lambda x: 60*parser.parse(x).hour + parser.parse(x).minute)
        
# sort events in chronological order
OMs = OMs.sort_values(['start_date', 'start_time'])


In [4]:
# function to convert minutes elapsed in day to hours and minutes string
def time_string(minutes):
    
    # hours elapsed in day
    hour = int(minutes/60)
    
    # if noon or after
    if hour > 12:
        if hour != 24:
            hour = f'{hour-12}:PM'
        else:
            hour = '12:AM'
    # if before noon
    else:
        if hour != 12:
            hour = f'{hour}:AM'
        else: hour = f'12:PM'
    
    # number of minutes into established hour of the day
    minute = minutes%60
    
    # if on the hour
    if minute == 0:
        hour = hour.replace(':', '')
    # if minutes into the hour
    else:
        hour = hour.replace(':', f':{minute}')
    
    return hour


In [5]:
# data frame of open mics before manipulation
OGs = OMs.copy()

# apply time string function to minutes elapsed in day
OGs['start_time'] = OGs['start_time'].apply(time_string)
OGs['end_time'] = OGs['end_time'].apply(time_string)

# display dataframe
OGs


,event_name,venue,neighborhood,start_date,start_time,end_date,end_time,all_day_event,category,event_cost,purchase_req,free,show_map_link,show_map,event_description,stage_time,event_website
90,Storytelling Mic,LA Wine,DTLA,2020-01-13,7:30PM,2020-01-13,9:30PM,False,Storytelling,$0,No min.,Free mics\n,True,True,\n\nin Suite 200\nFor storytelling only\nSign-...,5 min.,NaN
303,LA Connection Comedy Theater,LA Connection Comedy Theater,Burbank/NoHo/Valley,2020-01-16,6PM,2020-01-16,7PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nSign-up in advance at https://slotted.co/h...,5 min.,https://slotted.co/hotmedusa-
264,The Barkley,The Barkley,Pasadena,2020-01-22,9PM,2020-01-22,11PM,False,Open Mic,$0,No min.,Free mics\n,True,True,\n\nStarts 9PM\nRolling lottery\nHosted by Van...,3 min.,NaN
383,(7-9PM Hourly) The Open Mic Spot,The Open Mic Spot,South LA,2020-01-24,7PM,2020-01-24,9PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nSign-up in advance at https://openmicspot....,5 min.,https://openmicspot.com/open-mic/
447,(8-10PM Hourly) Therapeutic Noise (in Long Beach),Therapeutic Noise,Long Beach,2020-01-25,8PM,2020-01-25,11PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nSign-up in advance at https://slotted.co/t...,6 min.,https://slotted.co/therapeuticnoise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,Edendale Branch Library,Edendale Branch Library,Los Feliz/Silverlake/Echo Park,2020-04-02,6PM,2020-04-02,8PM,False,Open Mic,$0,No min.,Free mics\n,True,True,\n\nSign-up 5:45PM\nStarts 6PM\nEnds 7:30PM\nF...,5 min.,NaN
368,Improv Jam,UCB Sunset Inner Sanctum,Hollywood/East Hollywood,2020-04-03,6PM,2020-04-03,7PM,False,Open Mic,$0,No min.,Free mics\n,True,True,\n\nFor improv only\nStarts 6PM\nAnyone can play,NaN,NaN
290,Lyric Hyperion Theatre & Cafe,Lyric Hyperion Theatre & Cafe,Los Feliz/Silverlake/Echo Park,2020-04-04,4PM,2020-04-04,6PM,False,Open Mic,NaN,1 item min.,NaN,True,True,\n\nSign-up 4:30PM \nStarts 5PM \nEnds 7PM \nL...,4 min.,NaN
71,Character Spa,Lyric Hyperion Theatre & Cafe,Los Feliz/Silverlake/Echo Park,2020-04-06,4PM,2020-04-06,6PM,False,Character Mic,$0,No min.,Free mics\n,True,True,\n\nFor characters only\nThis mic is an opport...,7 min.,https://slotted.co/characterspa


In [6]:
# new columns to store recalculated event start and stop dates/times
OMs['new_start_time'] = OMs['start_time']
OMs['new_end_time'] = OMs['end_time']
OMs['new_start_date'] = OMs['start_date']
OMs['new_end_date'] = OMs['end_date']


# list to hold names of all unique events
events = OMs['event_name'].unique()

# iterate through all unique events
for e in events:

    # all events that share a particular name
    event = OMs.copy().loc[OMs['event_name'] == e,:]

    # list to hold the indicies of events of a shared name
    rows = list(event.index)

    # consolidate times
    new = 0
    start = event['start_time'][rows[0]]
    for i in range(1, len(rows)):
        end = event[f'end_time'][rows[i]]
        if event['start_time'][rows[i]] == event['end_time'][rows[i-1]]:
            OMs.loc[rows[i-1: i+1], 'new_start_time'] = start
            OMs.loc[rows[new:i+1], 'new_end_time'] = end
        else:
            new = i
            start = event['start_time'][rows[i]]

    # consolidate dates
    new = 0
    start = event['start_date'][rows[0]]
    for i in range(1, len(rows)):
        end = event[f'end_date'][rows[i]]
        if (event['start_date'][rows[i]] - event['end_date'][rows[i-1]]).days < 2:
            OMs.loc[rows[i-1: i+1], 'new_start_date'] = start
            OMs.loc[rows[new:i+1], 'new_end_date'] = end
        else:
            new = i
            start = event['start_date'][rows[i]]


In [7]:
# adjust times in event description to match new event start and stop times
OMs['event_description'] = OMs.apply(lambda x: x['event_description'].replace(time_string(x['start_time']), time_string(x['new_start_time'])), axis=1)
OMs['event_description'] = OMs.apply(lambda x: x['event_description'].replace(time_string(x['end_time']), time_string(x['new_end_time'])), axis=1)


In [8]:
# iterate through columns for begining and end times/dates
for bne in begin_and_end:
    
    # set start and stop columns to new start and stop columns
    OMs[bne] = OMs[f'new_{bne}']
    # drop new start and stop columns
    OMs = OMs.drop(columns=f'new_{bne}')

# convert values in time columns to stings that indicate hours and minutes of the day
OMs['start_time'] = OMs['start_time'].apply(time_string)
OMs['end_time'] = OMs['end_time'].apply(time_string)

# drop rows with the same information
OMs = OMs.drop_duplicates().reset_index(drop=True)


In [9]:
# display final dataframe
OMs


,event_name,venue,neighborhood,start_date,start_time,end_date,end_time,all_day_event,category,event_cost,purchase_req,free,show_map_link,show_map,event_description,stage_time,event_website
0,Storytelling Mic,LA Wine,DTLA,2020-01-13,7:30PM,2020-01-13,9:30PM,False,Storytelling,$0,No min.,Free mics\n,True,True,\n\nin Suite 200\nFor storytelling only\nSign-...,5 min.,NaN
1,LA Connection Comedy Theater,LA Connection Comedy Theater,Burbank/NoHo/Valley,2020-01-16,6PM,2020-01-16,7PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nSign-up in advance at https://slotted.co/h...,5 min.,https://slotted.co/hotmedusa-
2,The Barkley,The Barkley,Pasadena,2020-01-22,9PM,2020-01-22,11PM,False,Open Mic,$0,No min.,Free mics\n,True,True,\n\nStarts 9PM\nRolling lottery\nHosted by Van...,3 min.,NaN
3,(7-9PM Hourly) The Open Mic Spot,The Open Mic Spot,South LA,2020-01-24,7PM,2020-01-24,9PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nSign-up in advance at https://openmicspot....,5 min.,https://openmicspot.com/open-mic/
4,(8-10PM Hourly) Therapeutic Noise (in Long Beach),Therapeutic Noise,Long Beach,2020-01-25,8PM,2020-01-25,11PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nSign-up in advance at https://slotted.co/t...,6 min.,https://slotted.co/therapeuticnoise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,Edendale Branch Library,Edendale Branch Library,Los Feliz/Silverlake/Echo Park,2020-04-02,6PM,2020-04-02,8PM,False,Open Mic,$0,No min.,Free mics\n,True,True,\n\nSign-up 5:45PM\nStarts 6PM\nEnds 7:30PM\nF...,5 min.,NaN
276,Improv Jam,UCB Sunset Inner Sanctum,Hollywood/East Hollywood,2020-04-03,6PM,2020-04-03,7PM,False,Open Mic,$0,No min.,Free mics\n,True,True,\n\nFor improv only\nStarts 6PM\nAnyone can play,NaN,NaN
277,Lyric Hyperion Theatre & Cafe,Lyric Hyperion Theatre & Cafe,Los Feliz/Silverlake/Echo Park,2020-04-04,4PM,2020-04-04,6PM,False,Open Mic,NaN,1 item min.,NaN,True,True,\n\nSign-up 4:30PM \nStarts 5PM \nEnds 7PM \nL...,4 min.,NaN
278,Character Spa,Lyric Hyperion Theatre & Cafe,Los Feliz/Silverlake/Echo Park,2020-04-06,4PM,2020-04-06,6PM,False,Character Mic,$0,No min.,Free mics\n,True,True,\n\nFor characters only\nThis mic is an opport...,7 min.,https://slotted.co/characterspa


In [10]:
# event with most instances in original data set
OGs.loc[OGs['event_name'] == 'Echoes on Pico']


,event_name,venue,neighborhood,start_date,start_time,end_date,end_time,all_day_event,category,event_cost,purchase_req,free,show_map_link,show_map,event_description,stage_time,event_website
443,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-14,8PM,2020-03-14,9PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
454,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-14,9PM,2020-03-14,10PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
458,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-14,10PM,2020-03-14,11PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
104,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-16,8PM,2020-03-16,9PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
111,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-16,9PM,2020-03-16,10PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
116,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-16,10PM,2020-03-16,11PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
168,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-17,8PM,2020-03-17,9PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
191,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-17,9PM,2020-03-17,10PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
195,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-17,10PM,2020-03-17,11PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
243,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-18,8PM,2020-03-18,9PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/


In [11]:
# same event represented by new, consolidated data set
OMs.loc[OMs['event_name'] == 'Echoes on Pico']


,event_name,venue,neighborhood,start_date,start_time,end_date,end_time,all_day_event,category,event_cost,purchase_req,free,show_map_link,show_map,event_description,stage_time,event_website
176,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-14,8PM,2020-03-14,11PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
213,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-16,8PM,2020-03-20,11PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/


In [12]:
# display differences in length between datasets
print(f'Original data set length: {len(OGs)}')
print(f'New data set length: {len(OMs)}')
print(f'Percent reduction: {round(100 - 100*len(OMs)/len(OGs), 2)}')


Original data set length: 454
New data set length: 280
Percent reduction: 38.33
